Etude du modele en fonction des données d'entrainement et de test utilisé.

Etude sur 4, 6, 12 visit et 1 visit: 80% pour train, 20% pour test

Cas étudié :
- Données totale d'entraienement correspondent à la somme des donnée de chaque visit ajouté de manière desordonné : le split est réalisé après la sommation des visit
- Données d'une seule visite

Le nombre de données d'entraienement est le meme pour les cas étudié. Le jeu de donnée de test est le meme. Des test sont également réalisé sur d'autre visit

Comparaison avec accuracy et matrice de confusion, ROC curve et AUC


Cas étudié :
- Entrainement du model visit 90780 : save as /../model_90780.h5
- Entrainement du model visit 100764 : save as /../model_100764.h5
- Entrainement du model visit 101836 : save as /../model_101836.h5
- Entrainement du model visit 89762 : save as /../model_89762.h5
- Entrainement du model visit sur 4 visit 90780, 100764, 101836, 89762 : save as /../model_4_visit.h5
- Entrainement du model visit sur 6 visit 90780, 100764, 101836, 89762, 97582, 104516 : save as /../model_6_visit.h5
- Entrainement du model visit sur 12 visit 90780, 100764, 101836, 89762, 97582, 104516, 103076, 103166, 103712, 90752, 92502, 95084 : save as /../model_12_visit.h5

# Init python stuff

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns
import sys,os
from importlib import reload
from matplotlib_venn import venn2, venn3


from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

from keras_tuner import HyperModel
from keras_tuner.tuners import BayesianOptimization
from venny4py.venny4py import *
from collections import Counter
from matplotlib import pyplot as plt
from upsetplot import generate_counts, plot,from_contents

import pandas as pd
pd.set_option('display.max_columns', None)

from mpl_toolkits.axes_grid1 import make_axes_locatable

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #Permet d'enlever les Warning tensorflow

np.random.seed(1) # NumPy
import random
random.seed(2) # Python
tf.random.set_seed(3) # Tensorflow 

sys.path.append('..')
sys.path.append('/sps/lsst/users/rbonnetguerrini/ML4_transientV1/saved')

from utils import ImageDisplayer, datadir, rundir, pddir
from CNN_model import CNNHyperModel
from Retrieve_data import retrieve_data, retrieve_data_gen3
from df_bogus import trans_deg2, df_proba, spy

imagedir = f'{rundir}/Compare_visit/Figure/'
models = f'{rundir}/Compare_visit/models'

2024-10-09 15:05:08.509139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 15:05:10.261013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from evaluation import evaluating_model
multiple_band_visits = [95092, 91556, 101500, 104516]


In [3]:
import warnings; warnings.simplefilter('ignore')


In [4]:
#Allows you to display all the columns of the dataframe
pd.set_option('display.max_columns', None)


In [5]:
test_size = 0.2
nbre_data = 48093 # Nombre de donnée utilisé dans le cas d'une seule visite
nombre_data = int(nbre_data/4) # Nombre de donnée utilisé dans le cas de concatenation de 4 visite

In [6]:
from pathlib import Path
import re

# Specify the directory path
directory_path =Path(pddir)

# Regular expression to match the number in the CSV file names
pattern = re.compile(r'Cat_real_sources_visit_(\d+)\.csv')

# List to hold the extracted numbers
numbers = []

# Extract the numbers
for element in directory_path.iterdir():
    if element.is_file() and element.suffix == '.csv':
        match = pattern.search(element.name)
        if match:
            numbers.append(int(match.group(1)))

# Print the extracted numbers
print(len(numbers))

for visit_id in numbers: 
    path = os.path.join(datadir, f"Detected_real_sources_visit_{visit_id}.npy")
    if not os.path.exists(path): 
        print(visit_id)

51
105748
108242
89816
104180
95820
102128
97608
105560
110398


# Test with Gen3 data

## create my own pipeline

In [5]:
data_322 = retrieve_data_gen3(visits=[322, 346], rotate = True)
df_train_322, df_test_322, x_train_322, x_test_322, y_train_322, y_test_322 = data_322.data(test_size=test_size)


In [6]:
df_train_322

,y_train,diaSourceId,visit,band,apFlux,apFluxErr,snr
0,1,138388141244506,322,y,666.896505,678.177843,5.131612
1,0,148711095140438,346,y,-2343.878753,698.212791,5.677313
2,0,148693915271176,346,y,-113.80533,716.080019,8.881531
3,0,148730422493247,346,y,-225.311468,691.70138,5.598454
4,1,138403173629970,322,y,1821.701151,663.690714,6.763751
...,...,...,...,...,...,...,...
3630,0,148730422493282,346,y,-2748.86043,660.782947,6.210135
3631,0,138405321113702,322,y,-1615.231398,679.988963,9.0687
3632,0,148696062754826,346,y,-795.76414,692.847583,7.999061
3633,0,148711095140451,346,y,-531.848128,707.465804,14.95161


In [7]:
df_test_322

,y_test,diaSourceId,visit,band,apFlux,apFluxErr,snr
0,0,138388141244446,322,y,2210.287179,735.392196,18.324314
1,0,148730422493311,346,y,-1659.536969,701.672086,10.589653
2,0,138422500982804,322,y,-580.901425,690.338025,8.012455
3,0,138405321113620,322,y,1421.108131,710.262995,15.39761
4,0,138385993760815,322,y,3327.313003,637.991519,7.67599
...,...,...,...,...,...,...,...
904,0,148730422493270,346,y,-2456.184541,671.707506,5.273296
905,1,148696062754913,346,y,344264.523288,699.66263,940.796082
906,0,148713242624075,346,y,4484.900693,663.898465,11.526451
907,0,148706800173090,346,y,84378.273909,846.111363,210.629715


In [8]:
# Recovery of normalize data and dataframe which contains features
data_322 = retrieve_data_gen3(visits=322, rotate = True)
df_train_322, df_test_322, x_train_322, x_test_322, y_train_322, y_test_322 = data_322.data(test_size=test_size)


In [9]:
print(x_train_322.shape)
print(x_test_322.shape)
print(y_train_322.shape)
print(y_test_322.shape)


(1811, 30, 30, 1)
(453, 30, 30, 1)
(1811,)
(453,)


In [12]:
y_test_322

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0])

In [3]:
all_visits = [322, 346, 358, 1178, 1184, 1204, 1206, 1214, 1220, 1242, 1248, 11690,
    11694, 11696, 11698, 11704, 11710, 11724, 11738, 11740, 17900, 17904,
    17906, 17926, 17948, 17950, 19680, 19684, 19694, 19696, 22632, 22662,
    23694, 23704, 23706, 23718, 29336, 29350, 30482, 30490]

In [10]:
from utils import All_visits_gen3
data_all_visits = retrieve_data_gen3(visits=All_visits_gen3, rotate =True)
df_train_all_visits, df_test_all_visits, x_train_all_visits, x_test_all_visits, y_train_all_visits, y_test_all_visits = data_all_visits.data(test_size=test_size)


In [11]:
print(x_train_all_visits.shape)
print(x_test_all_visits.shape)
print(y_train_all_visits.shape)
print(y_test_all_visits.shape)

(185296, 30, 30, 1)
(46324, 30, 30, 1)
(185296,)
(46324,)


## Training 

In [12]:
INPUT_SHAPE = (30,30,1) # Size of data
NUM_CLASSES = 2 # Number of differente classe in data

In [13]:
#Tuner's settings
MAX_EPOCHS = 50
MAX_TRIALS = 1
EXECUTION_PER_TRIAL = 5

In [14]:
# Create the CNNHyperModel model
model_all_visit = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, max_epochs=MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_all_visits, x_test=x_test_all_visits, y_train=y_train_all_visits, y_test=y_test_all_visits)

In [15]:
#Train the CNNHyperModel model
best_model, history = model_all_visit.Entrainement_bayes()

Trial 1 Complete [00h 53m 03s]
val_accuracy: 0.9640316128730774

Best val_accuracy So Far: 0.9640316128730774
Total elapsed time: 00h 53m 03s
Epoch 1/50


2024-10-09 16:00:05.821999: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1448/1448 [==============================] - 19s 11ms/step - loss: 0.1894 - accuracy: 0.9206 - val_loss: 0.1357 - val_accuracy: 0.9463
Epoch 2/50
1448/1448 [==============================] - 14s 10ms/step - loss: 0.1404 - accuracy: 0.9444 - val_loss: 0.1196 - val_accuracy: 0.9538
Epoch 3/50
1448/1448 [==============================] - 14s 10ms/step - loss: 0.1290 - accuracy: 0.9496 - val_loss: 0.1119 - val_accuracy: 0.9586
Epoch 4/50
1448/1448 [==============================] - 13s 9ms/step - loss: 0.1235 - accuracy: 0.9527 - val_loss: 0.1090 - val_accuracy: 0.9592
Epoch 5/50
1448/1448 [==============================] - 13s 9ms/step - loss: 0.1214 - accuracy: 0.9537 - val_loss: 0.1138 - val_accuracy: 0.9555
Epoch 6/50
1448/1448 [==============================] - 14s 9ms/step - loss: 0.1187 - accuracy: 0.9548 - val_loss: 0.1057 - val_accuracy: 0.9612
Epoch 7/50
1448/1448 [==============================] - 14s 9ms/step - loss: 0.1175 - accuracy: 0.9555 - val_loss: 0.1028 - val_accuracy: 

2024-10-09 16:11:16.678853: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


In [ ]:

best_model.save('ML4_transientV3/Compare_visit/models/model_all_gen3_rotate.h5')

In [ ]:
ig, axes = plt.subplots(2, 1, figsize=(10, 10))

for i, hist in enumerate(history):
    # Plot loss
    axes[0].plot(hist['loss'], label=f'Trial {i+1} - Training Loss')
    axes[0].plot(hist['val_loss'], '--', label=f'Trial {i+1} - Validation Loss')

    # Plot accuracy
    axes[1].plot(hist['accuracy'], label=f'Trial {i+1} - Training Accuracy')
    axes[1].plot(hist['val_accuracy'], '--', label=f'Trial {i+1} - Validation Accuracy')

axes[0].set_title('Loss over epochs')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

axes[1].set_title('Accuracy over epochs')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

## Evaluate the model 

In [ ]:
hypermodel_gen3 = tf.keras.models.load_model(f'{rundir}/Compare_visit/models/model_all_gen3.h5')

In [ ]:

evaluation = evaluating_model(model='model_all_gen3', visit=multiple_band_visits)


In [ ]:
evaluation.full_evaluation_plot()

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_322 = retriev_data(visits=322)
df_train_322, df_test_322, x_train_322, x_test_322, y_train_322, y_test_322 = data_322.data(test_size=test_size)


In [ ]:
#Evaluate the model on testing set
score_best_model_global_all_visit = hypermodel_90780.evaluate(x_test_322, y_test_322, verbose=1)

y_sigmoid_all_visit_322 = hypermodel_90780.predict(x_test_322) #Table de probabilité
df_test_322['y_pred_best_model_all_visit'] = np.argmax(y_sigmoid_all_visit_322, axis=-1) #Permet de recuperer la classe d'apartenance
score_best_model_global_all_visit

In [ ]:
from utils import datadir
visit = 322

In [ ]:
data = np.load(os.path.join(datadir,f"cutouts/Detected_obj_sources_visit_{visit}.npy"))
data[1]

In [ ]:
#Evaluate the model on testing set
score_90780 = best_model.evaluate(x_test_90780, y_test_90780, verbose=1)

y_sigmoid_90780 = hypermodel_90780.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred'] = np.argmax(y_sigmoid_90780, axis=-1) #Permet de recuperer la classe d'apartenance
score_90780

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_90780_93404 = hypermodel_90780.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_90780_93404 = hypermodel_90780.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_model_90780'] = np.argmax(y_sigmoid_90780_93404, axis=-1) #Permet de recuperer la classe d'apartenance

# Retrieve data

## Chargement des données pour etude sur plusieurs visit

Les données d'entrainement sont ajouté les unes apres les autres, ils n'y a pas de melande entre les differentes visites

In [ ]:
nbre_visit_4=[90780, 100764, 101836, 89762]


In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_visit_4, nbre_data = nbre_data)
df_train_4_visit, df_test_4_visit, x_train_4_visit, x_test_4_visit, y_train_4_visit, y_test_4_visit = data.donnees(test_size=test_size)


In [ ]:
nbre_visit_6=[90780, 100764, 101836, 89762, 97582, 104516]


In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_visit_6, nbre_data = nbre_data)
df_train_6_visit, df_test_6_visit, x_train_6_visit, x_test_6_visit, y_train_6_visit, y_test_6_visit = data.donnees(test_size=test_size)

In [ ]:
nbre_visit_12=[90780, 100764, 101836, 89762, 97582, 104516, 103076, 103166, 103712, 90752, 92502, 95084]


In [ ]:
# Recovery of normalize data and dataframe which contains features
data_12_visit = retrieve_data(nbre_visit=nbre_visit_12, nbre_data = nbre_data)
df_train_12_visit, df_test_12_visit, x_train_12_visit, x_test_12_visit, y_train_12_visit, y_test_12_visit = data_12_visit.donnees(test_size=test_size)


In [ ]:
#
nbre_visit_42 = [91922, 103076, 114502, 102426, 92502, 93404, 106144, 106532, 95084, 96898, 103712, 107882, 104856, 97026, 111352, 113142, 100824, 111426, 113476, 95148, 90780, 104516, 91898, 96222, 91556, 93410, 106078, 113176, 97582, 89762, 105108, 101500, 101836, 103166, 106342, 107976, 95092, 100764, 104618, 108216, 90752, 108692]

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_42_visit = retrieve_data(nbre_visit=nbre_visit_42, nbre_data = nbre_data)
df_train_42_visit, df_test_42_visit, x_train_42_visit, x_test_42_visit, y_train_42_visit, y_test_42_visit = data_42_visit.donnees(test_size=test_size)


In [ ]:
print(len(df_train_42_visit))
print(len(df_train_4_visit))

## Chargement données pour etude visit par visit

In [14]:
# Recovery of normalize data and dataframe which contains features
data_90780 = retrieve_data(nbre_visit=90780, nbre_data = nbre_data)
df_train_90780, df_test_90780, x_train_90780, x_test_90780, y_train_90780, y_test_90780 = data_90780.donnees(test_size=test_size)


In [15]:
print(x_train_90780.shape)
print(x_test_90780.shape)
print(y_train_90780.shape)
print(y_test_90780.shape)



(42690, 30, 30, 1)
(10673, 30, 30, 1)
(42690,)
(10673,)


In [9]:
x_test_90780.type

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_100764 = retrieve_data(nbre_visit=100764, nbre_data = nbre_data)
df_train_100764, df_test_100764, x_train_100764, x_test_100764, y_train_100764, y_test_100764 = data_100764.donnees(test_size=test_size)


In [ ]:
# Recovery of normalize data and dataframe which contains features
data_101836 = retrieve_data(nbre_visit=101836, nbre_data = nbre_data)
df_train_101836, df_test_101836, x_train_101836, x_test_101836, y_train_101836, y_test_101836 = data_101836.donnees(test_size=test_size)


In [ ]:
# Recovery of normalize data and dataframe which contains features
data_89762 = retrieve_data(nbre_visit=89762, nbre_data = nbre_data)
df_train_89762, df_test_89762, x_train_89762, x_test_89762, y_train_89762, y_test_89762 = data_89762.donnees(test_size=test_size)


# Test data

Données de test utilisé commune a tout les model : visit 93404

In [16]:
# Recovery of normalize data and dataframe which contains features
data_93404 = retrieve_data(nbre_visit=93404)
df_train_93404, df_test_93404, x_train_93404, x_test_93404, y_train_93404, y_test_93404 = data_93404.donnees(test_size=test_size)

# Parametre model

In [17]:
INPUT_SHAPE = (30,30,1) # Size of data
NUM_CLASSES = 2 # Number of differente classe in data

In [20]:
#Tuner's settings
MAX_EPOCHS = 20
MAX_TRIALS = 5
EXECUTION_PER_TRIAL = 2

# Entrainement du model avec toute les données melangé 4 visits

In [ ]:
# Create the CNNHyperModel model
model_4_visit = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_4_visit, x_test=x_test_4_visit, y_train=y_train_4_visit, y_test=y_test_4_visit)


In [ ]:
#Train the CNNHyperModel model
#hypermodel_4_visit, history_4_visit = model_4_visit.Entrainement_bayes()

In [ ]:
#hypermodel_4_visit.save(f'{rundir}/Compare_visit/models/model_4_visit.h5')

In [ ]:
hypermodel_4_visit = tf.keras.models.load_model(f'{rundir}/Compare_visit/models/model_4_visit.h5')

In [ ]:
model_ = 'model_4_visit'
evaluation = evaluating_model(model=model_, visit=multiple_band_visits)


In [ ]:
evaluation.full_evaluation_plot()

In [ ]:
#Evaluate the model on testing set
score_best_model_global_4_visit = hypermodel_4_visit.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_4_visit_93404 = hypermodel_4_visit.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_4_visit'] = np.argmax(y_sigmoid_4_visit_93404, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_best_model_global_4_visit_90780 = hypermodel_4_visit.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_4_visit_90780 = hypermodel_4_visit.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_4_visit_90780'] = np.argmax(y_sigmoid_4_visit_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_best_model_global_4_visit_101836 = hypermodel_4_visit.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_4_visit_101836 = hypermodel_4_visit.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_4_visit_101836'] = np.argmax(y_sigmoid_4_visit_101836, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model avec toute les données melangé 6 visits

In [ ]:
# Create the CNNHyperModel model
model_6_visit = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_6_visit, x_test=x_test_6_visit, y_train=y_train_6_visit, y_test=y_test_6_visit)


In [ ]:
#Train the CNNHyperModel model
#hypermodel_6_visit, history_6_visit = model_6_visit.Entrainement_bayes()

In [ ]:
#hypermodel_6_visit.save(f'{rundir}/Compare_visit/models/model_6_visit.h5')

In [ ]:
hypermodel_6_visit = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit = hypermodel_6_visit.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_93404 = hypermodel_6_visit.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_6_visit'] = np.argmax(y_sigmoid_6_visit_93404, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_best_model_global_6_visit_90780 = hypermodel_6_visit.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_90780 = hypermodel_6_visit.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_6_visit_90780'] = np.argmax(y_sigmoid_6_visit_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_best_model_global_6_visit_101836 = hypermodel_6_visit.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_101836 = hypermodel_6_visit.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_6_visit_101836'] = np.argmax(y_sigmoid_6_visit_101836, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model avec toute les données melangé 12 visits

In [ ]:
# Create the CNNHyperModel model
model_12_visit = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_12_visit, x_test=x_test_12_visit, y_train=y_train_12_visit, y_test=y_test_12_visit)


In [ ]:
#Train the CNNHyperModel model
hypermodel_12_visit, history_12_visit = model_12_visit.Entrainement_bayes()

In [ ]:
#hypermodel_12_visit.save(f'{rundir}/Compare_visit/models/model_12_visit.h5')

In [ ]:
hypermodel_12_visit = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_12_visit.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_12_visit = hypermodel_12_visit.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_12_visit = hypermodel_12_visit.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_12_visit'] = np.argmax(y_sigmoid_12_visit, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_12_visit_90780 = hypermodel_12_visit.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_12_visit_90780 = hypermodel_12_visit.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_12_visit_90780'] = np.argmax(y_sigmoid_12_visit_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_12_visit_101836 = hypermodel_12_visit.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_12_visit_101836 = hypermodel_12_visit.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_12_visit_101836'] = np.argmax(y_sigmoid_12_visit_101836, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model avec toute les données melangé 42 visits

In [ ]:
# Create the CNNHyperModel model
model_42_visit = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_42_visit, x_test=x_test_42_visit, y_train=y_train_42_visit, y_test=y_test_42_visit)


In [ ]:
#Train the CNNHyperModel model
hypermodel_42_visit, history_42_visit = model_42_visit.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
hypermodel_42_visit.save(f'{rundir}/Compare_visit/models/model_42_visit.h5')

In [ ]:
hypermodel_42_visit = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_42_visit.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_42_visit = hypermodel_42_visit.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_42_visit = hypermodel_42_visit.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_42_visit'] = np.argmax(y_sigmoid_42_visit, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_42_visit_90780 = hypermodel_42_visit.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_42_visit_90780 = hypermodel_42_visit.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_42_visit_90780'] = np.argmax(y_sigmoid_42_visit_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_42_visit_101836 = hypermodel_42_visit.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_42_visit_101836 = hypermodel_42_visit.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_42_visit_101836'] = np.argmax(y_sigmoid_42_visit_101836, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 90780

In [21]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, max_epochs=MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_90780, x_test=x_test_90780, y_train=y_train_90780, y_test=y_test_90780)

In [22]:
#Train the CNNHyperModel model
hypermodel, history = model.Entrainement_bayes()

(None, 14, 14, 8)

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
21                |21                |filters_1
0.15              |0.15              |dropout_1
14                |14                |filters_2
0.45              |0.45              |dropout_2
1616              |1616              |units
0.25              |0.25              |dropout_3
0.001             |0.001             |learning_rate

(None, 14, 14, 21)
Epoch 1/20
334/334 [==============================] - 10s 28ms/step - loss: 0.3393 - accuracy: 0.8413 - val_loss: 0.2434 - val_accuracy: 0.8948
Epoch 2/20
334/334 [==============================] - 9s 27ms/step - loss: 0.2428 - accuracy: 0.8927 - val_loss: 0.2052 - val_accuracy: 0.9122
Epoch 3/20
334/334 [==============================] - 9s 26ms/step - loss: 0.2163 - accuracy: 0.9062 - val_loss: 0.1889 - val_accuracy: 0.9189
Epoch 4/20
334/334 [==============================] - 8s 25ms/step - loss: 0.2019 - accuracy: 0.9154 - val_loss: 0.1

KeyboardInterrupt: 

In [ ]:
plot_metrics(history)

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_90780.h5')

In [ ]:
#ypermodel_90780 = hypermodel
hypermodel_90780 = tf.keras.models.load_model(f'{rundir}/Compare_visit/models/model_90780.h5')

In [ ]:
#Evaluate the model on testing set
score_90780 = hypermodel_90780.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_90780 = hypermodel_90780.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred'] = np.argmax(y_sigmoid_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
print(df_test_90780['y_pred'])

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_90780_93404 = hypermodel_90780.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_90780_93404 = hypermodel_90780.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_model_90780'] = np.argmax(y_sigmoid_90780_93404, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 100764

In [ ]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_100764, x_test=x_test_100764, y_train=y_train_100764, y_test=y_test_100764)


In [ ]:
#Train the CNNHyperModel model
#hypermodel, history = model.Entrainement_bayes()

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_100764.h5')

In [ ]:
hypermodel_100764 = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_100764.h5')

In [ ]:
#Evaluate the model on testing set
score_100764 = hypermodel_100764.evaluate(x_test_100764, y_test_100764, verbose=0)

y_sigmoid_100764 = hypermodel_100764.predict(x_test_100764) #Table de probabilité
df_test_100764['y_pred'] = np.argmax(y_sigmoid_100764, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_100764_93404 = hypermodel_100764.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_100764_93404 = hypermodel_100764.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_model_100764'] = np.argmax(y_sigmoid_100764_93404, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 101836

In [ ]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_101836, x_test=x_test_101836, y_train=y_train_101836, y_test=y_test_101836)


In [ ]:
#Train the CNNHyperModel model
#hypermodel, history = model.Entrainement_bayes()

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_101836.h5')

In [ ]:
hypermodel_101836 = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_101836.h5')

In [ ]:
#Evaluate the model on testing set
score_101836 = hypermodel_101836.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_101836 = hypermodel_101836.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred'] = np.argmax(y_sigmoid_101836, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_101836_93404 = hypermodel_101836.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_101836_93404 = hypermodel_101836.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_model_101836'] = np.argmax(y_sigmoid_101836_93404, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 89762

In [ ]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_89762, x_test=x_test_89762, y_train=y_train_89762, y_test=y_test_89762)


In [ ]:
#Train the CNNHyperModel model
#hypermodel, history = model.Entrainement_bayes()

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_89762.h5')

In [ ]:
hypermodel_89762 = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_89762.h5')

In [ ]:
#Evaluate the model on testing set
score_89762 = hypermodel_89762.evaluate(x_test_89762, y_test_89762, verbose=0)

y_sigmoid_89762 = hypermodel_89762.predict(x_test_89762) #Table de probabilité
df_test_89762['y_pred'] = np.argmax(y_sigmoid_89762, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 93404 in order to compare with train on 4 visit
score_89762_93404 = hypermodel_89762.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_89762_93404 = hypermodel_89762.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_model_89762'] = np.argmax(y_sigmoid_89762_93404, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 89762 test sur visit 101836

In [ ]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_89762, x_test=x_test_101836, y_train=y_train_89762, y_test=y_test_101836)


In [ ]:
#Train the CNNHyperModel model
#hypermodel, history = model.Entrainement_bayes()

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_train_89762_test_101836.h5')

In [ ]:
hypermodel = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_train_89762_test_101836.h5')

In [ ]:
#Evaluate the model on testing set
score_89762_101836 = hypermodel.evaluate(x_test_101836, y_test_101836, verbose=0)

In [ ]:
y_sigmoid = hypermodel.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_train_101836'] = np.argmax(y_sigmoid, axis=-1) #Permet de recuperer la classe d'apartenance

# Entrainement du model visit 101836 test sur visit 100764

In [ ]:
# Create the CNNHyperModel model
model = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                      x_train = x_train_101836, x_test=x_test_100764, y_train=y_train_101836, y_test=y_test_100764)


In [ ]:
#Train the CNNHyperModel model
#hypermodel, history = model.Entrainement_bayes()

In [ ]:
#hypermodel.save(f'{rundir}/Compare_visit/models/model_train_101836_test_100764.h5')

In [ ]:
hypermodel = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_train_101836_test_100764.h5')

In [ ]:
#Evaluate the model on testing set
score_101836_100764= hypermodel.evaluate(x_test_100764, y_test_100764, verbose=0)

In [ ]:
y_sigmoid = hypermodel.predict(x_test_100764) #Table de probabilité
df_test_100764['y_pred_train_101836'] = np.argmax(y_sigmoid, axis=-1) #Permet de recuperer la classe d'apartenance

# Resultats

## Train sur 6 visit et 1 visite, test sur la meme visit

L'entrainement est realisé sur 6 visite dans le cas ou les données sont melangé et dans le cas ou les données ne sont pas melangé. L'entrainement est egalement realisé sur 1 seule visite.

Le test est realisé sur la visite 93404.

In [ ]:
#Train, test sur les 4 visite melangé et prediction sur la visite 93404

print(f'Test loss     : {score_best_model_global_6_visit[0]:4.4f}')
print(f'Test accuracy : {score_best_model_global_6_visit[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_93404, df_test_93404.y_pred_best_model_90780_global_split, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


In [ ]:
#Train, test sur la visite 90780 et prediction sur la visite 93404

print(f'Test loss     : {score_90780_93404[0]:4.4f}')
print(f'Test accuracy : {score_90780_93404[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_93404, df_test_93404.y_pred_best_model_90780_global_split, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


## Train sur 42 visit et 1 visit, test sur la meme visit

In [ ]:
#Train, test sur les 4 visite melangé et prediction sur la visite 93404

print(f'Test loss     : {score_best_model_global_42_visit[0]:4.4f}')
print(f'Test accuracy : {score_best_model_global_42_visit[1]:4.4f}')

disp = ConfusionMatrixDisplay.from_predictions(y_test_93404, df_test_93404.y_pred_best_model_global_42_visit, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='YlOrRd' )


## Visit by visit

In [ ]:
#Train, test et prediction sur la visite 90780
print(f'Test loss     : {score_90780[0]:4.4f}')
print(f'Test accuracy : {score_90780[1]:4.4f}')

disp = ConfusionMatrixDisplay.from_predictions(y_test_90780, df_test_90780.y_pred, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


In [ ]:
#Train, test et prediction sur la visite 100764
print(f'Test loss     : {score_100764[0]:4.4f}')
print(f'Test accuracy : {score_100764[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_100764, df_test_100764.y_pred, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


In [ ]:
#Train, test et prediction sur la visite 101836

print(f'Test loss     : {score_101836[0]:4.4f}')
print(f'Test accuracy : {score_101836[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_101836, df_test_101836.y_pred, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


In [ ]:
#Train, test et prediction sur la visite 89762

print(f'Test loss     : {score_89762[0]:4.4f}')
print(f'Test accuracy : {score_89762[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_89762, df_test_89762.y_pred, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


## Train on one visit test on other one

In [ ]:
#Train, test sur la visite 89762 et prediction sur la visite 101836

print(f'Test loss     : {score_89762_101836[0]:4.4f}')
print(f'Test accuracy : {score_89762_101836[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_101836, df_test_101836.y_pred_train_101836, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


In [ ]:
#Train, test sur la visite 101836 et prediction sur la visite 100764

print(f'Test loss     : {score_101836_100764[0]:4.4f}')
print(f'Test accuracy : {score_101836_100764[1]:4.4f}')

#disp = ConfusionMatrixDisplay.from_predictions(y_test_100764, df_test_100764.y_pred_train_101836, display_labels = ['bogus', 'transient'],normalize='true', colorbar=False, cmap='Blues' )


## Plot global

In [ ]:
name_1 =['42 visit','12 visit', '6 visit', '4 visit', 'visit 90780', 'visit 100764', 'visit 101836', 'visit 89762']
y_1=[score_best_model_global_42_visit[1],
    score_best_model_global_12_visit[1], 
     score_best_model_global_6_visit[1],
     score_best_model_global_4_visit[1], 
     score_90780_93404[1],
     score_100764_93404[1], 
     score_101836_93404[1], 
     score_89762_93404[1]]

name_2 =['visit 90780', 'visit 100764', 'visit 101836', 'visit 89762']

y_2=[score_90780[1], score_100764[1], score_101836[1], score_89762[1]]

name_3 = ['42 visit', '12 visit', '6 visit', '4 visit']

y_3 = [score_best_model_global_42_visit_90780[1],score_best_model_global_12_visit_90780[1], score_best_model_global_6_visit_90780[1], score_best_model_global_4_visit_90780[1]]
       
name_4 = ['42 visit', '12 visit', '6 visit', '4 visit']
       
y_4 = [score_best_model_global_42_visit_101836[1], score_best_model_global_12_visit_101836[1], score_best_model_global_6_visit_101836[1], score_best_model_global_4_visit_101836[1]]


In [ ]:
# Créer le diagramme à barres
fig, ax = plt.subplots(figsize=(8, 3), dpi=600)
ax.tick_params(axis='both', which='major', labelsize=6)

ax.grid(True, alpha=0.2) 
ax.scatter(name_1, y_1, color='blue', label='test visit 93404')
ax.scatter(name_2, y_2, color='red', label='test same visit')
ax.scatter(name_3, y_3, color='green', label='test visit 90780')
ax.scatter(name_4, y_4, color='orange', label='test visit 101836')


# Personnaliser l'apparence
ax.set_ylabel('Accuracy', fontsize=6)
ax.set_xticks(name_1 )
ax.legend(loc='lower left', fontsize=6)
#ax.set_ylim([0, 1])

fig.subplots_adjust(right=0.8)

# Afficher le diagramme
#plt.savefig('/sps/lsst/users/mguy/bogus_detection/bogus_detection_v7/Train_model/Figure_train_model/Comparaison_accuracy_train_model.png', dpi=400)
plt.show()


In [ ]:
df_train_90780['SNR_base_CircularApertureFlux_4_5']

# ROC Curve

Computation of the ROC curve (receiver operating characteristics) and the AUC (area under the curve)

AUC : 

* represent the degree of mesure of separability
* how much the model is capable of distinguish between classes
* higher the AUC is, better the model is

Compute :

FPR : False positive ratio; proportion of the bogus class got incorrectly classified

TPR : True positive ratio; proportion of the transient class got correctly classified


<br>
<br>

\begin{equation}
    FPR = \frac{FP}{TN + FP} \qquad TPR = \frac{TP}{TP + FN}
\end{equation}

## Calcul

In [ ]:
y_sigmoid_model = [y_sigmoid_42_visit, y_sigmoid_12_visit, y_sigmoid_6_visit_93404, y_sigmoid_90780_93404,
                   y_sigmoid_100764_93404, y_sigmoid_101836_93404,
                   y_sigmoid_89762_93404]

column_model = ["42_visit", "12_visit", "6_visit", "visit_90780", "visit_100764",
                   "visit_101836", "visit_89762"]



y_sigmoid_model_2=[y_sigmoid_90780, y_sigmoid_100764,
                   y_sigmoid_101836, y_sigmoid_89762]

column_model_2 =[ 'visit_90780', 'visit_100764', 'visit_101836', 'visit_89762']

y_test_2 = [y_test_90780, y_test_100764, y_test_101836, y_test_89762]



column_model_3 = ['42_visit', '12_visit', '6_visit']

y_sigmoid_model_3 = [y_sigmoid_42_visit_90780, y_sigmoid_12_visit_90780, y_sigmoid_6_visit_90780]


In [ ]:
def ROC( y_sigmoid, y_test):
    proba = np.arange(0.05, 1, 0.01)
    TPR = []
    FPR = []
    for j in proba:
        classe = []
        for i in y_sigmoid[:,1]:
            if i<j:
                classe.append(0)
            else:
                classe.append(1)

        CM = confusion_matrix(y_test, classe)
        TPR.append(CM[1,1]/(CM[1,1]+CM[1,0]))
        FPR.append(CM[0,1]/(CM[0,0]+CM[0,1]))
    AUC = auc(FPR, TPR)
    return TPR, FPR, AUC

In [ ]:
# Calcule la courbe ROC et l'AUC dans le cas du test sur la visit 93404
TPR_ = {}
FPR_ = {}
AUC_ = {}

for k, l in zip(y_sigmoid_model, column_model):
    TPR, FPR, AUC = ROC(k, y_test_93404)
    TPR_[l]=TPR
    FPR_[l]=FPR
    AUC_[l]=AUC

In [ ]:
# Calcule la courbe ROC et l'AUC dans le cas du test sur la meme visit que celle du train
TPR_2_ = {}
FPR_2_ = {}
AUC_2_ = {}

for k, l, m in zip(y_sigmoid_model_2, column_model_2, y_test_2):
    TPR, FPR, AUC = ROC( k, m)
    TPR_2_[l]=TPR
    FPR_2_[l]=FPR
    AUC_2_[l]=AUC

In [ ]:
# Calcule la courbe ROC et l'AUC dans le cas du test sur la meme visit que celle du train
TPR_3_ = {}
FPR_3_ = {}
AUC_3_ = {}

for k, l in zip(y_sigmoid_model_3, column_model_3):
    TPR, FPR, AUC = ROC( k, y_test_90780)
    TPR_3_[l]=TPR
    FPR_3_[l]=FPR
    AUC_3_[l]=AUC

## Diagramme

In [ ]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p = figure(x_axis_label='FPR', y_axis_label='TPR',tools=TOOLS, width=800, height=400)
colors = ['darkorchid','brown','darkcyan','black','orange', 'green', "olive", "mediumvioletred", "gray", "pink", "brown"]


for l, m in zip(column_model, colors):
    p.scatter(FPR_[l], TPR_[l], color=m, size=8, legend_label=f'threshold_{l}')

p.add_layout(p.legend[0], 'right')

p.legend.title = 'Model'

show(p)

In [ ]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p2 = figure(x_axis_label='FPR', y_axis_label='TPR',tools=TOOLS, width=800, height=400)
colors = ['darkorchid','brown','darkcyan','black','orange', 'green', "olive", "mediumvioletred", "gray", "pink", "brown"]


for l, m in zip(column_model, colors):
    p2.line(FPR_[l], TPR_[l], color=m, legend_label=f'threshold_{l}')

p2.add_layout(p2.legend[0], 'right')

p2.legend.title = 'Model'

show(p2)

In [ ]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p3 = figure(title="AUC",x_range=column_model,tools=TOOLS, width=800, height=400)

for l in column_model:
    p3.scatter(x=[l], y=[AUC_[l]], color='blue',size=8, legend_label=f'visit 93404')

for m in column_model_2:  
    p3.scatter([m], [AUC_2_[m]], color='red',size=8, legend_label=f'test same visit')

for n in column_model_3:  
    p3.scatter([n], [AUC_3_[n]], color='orange',size=8, legend_label=f'visit 90780')

p3.add_layout(p3.legend[0], 'right')

p3.legend.title = 'Model'

show(p3)

# Band analysis training 

In [ ]:
filt_dict={90752: 'g',
 95148: 'g',
 97582: 'g',
 101500: 'g',
 103076: 'g',
 104856: 'g',
 106078: 'g',
 107882: 'g',
 113176: 'g',
 91556: 'r',
 93404: 'r',
 96898: 'r',
 101836: 'r',
 103166: 'r',
 105108: 'r',
 106532: 'r',
 108692: 'r',
 111352: 'r',
 90780: 'i',
 91898: 'i',
 93410: 'i',
 95092: 'i',
 97026: 'i',
 100764: 'i',
 103712: 'i',
 104618: 'i',
 106342: 'i',
 108216: 'i',
 113142: 'i',
 113476: 'i',
 89762: 'z',
 91922: 'z',
 92502: 'z',
 95084: 'z',
 96222: 'z',
 100824: 'z',
 102426: 'z',
 104516: 'z',
 106144: 'z',
 107976: 'z',
 111426: 'z',
 114502: 'z',
 89816: 'y',
 95820: 'y',
 97608: 'y',
 102128: 'y',
 104180: 'y',
 105560: 'y',
 105748: 'y',
 108242: 'y',
 110398: 'y'}


[90780 i, 100764 i, 101836 r, 89762 z, 93404 r]

## G band

In [ ]:
nbre_6_visit_g=[95148, 97582, 101500, 103076, 104856, 106078, 107882]




In [ ]:
#Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_6_visit_g, nbre_data = nbre_data)
df_train_6_visit_g, df_test_6_visit_g, x_train_6_visit_g, x_test_6_visit_g, y_train_6_visit_g, y_test_6_visit_g = data.donnees(test_size=test_size)

In [ ]:
#Test set of the same band
#Recovery of normalize data and dataframe which contains features
data_90752 = retrieve_data(nbre_visit=90752)
df_train_90752, df_test_90752, x_train_90752, x_test_90752, y_train_90752, y_test_90752 = data_90752.donnees(test_size=test_size)

In [ ]:
# Create the CNNHyperModel model
model_6_visit_g = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_6_visit_g, x_test=x_test_6_visit_g, y_train=y_train_6_visit_g, y_test=y_test_6_visit_g)


In [ ]:
#Train the CNNHyperModel model
#hypermodel_6_visit_g, history_6_visit_g = model_6_visit_g.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
#hypermodel_6_visit_g.save(f'{rundir}/Compare_visit/models/model_6_visit_g.h5')

In [ ]:
hypermodel_6_visit_g = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit_g.h5')

In [ ]:
#Evaluate the model on testing set of the same band 
score_best_model_global_6_visit_g = hypermodel_6_visit_g.evaluate(x_test_90752, y_test_90752, verbose=0)

y_sigmoid_6_visit_g = hypermodel_6_visit_g.predict(x_test_90752) #Table de probabilité
df_test_90752['y_pred_best_model_global_6_visit_g'] = np.argmax(y_sigmoid_6_visit_g, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_g = hypermodel_6_visit_g.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_g = hypermodel_6_visit_g.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_6_visit_g'] = np.argmax(y_sigmoid_6_visit_g, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_6_visit_g_90780 = hypermodel_6_visit_g.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_g_90780 = hypermodel_6_visit_g.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_g_90780'] = np.argmax(y_sigmoid_6_visit_g_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_6_visit_g_101836 = hypermodel_6_visit_g.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_g_101836 = hypermodel_6_visit_g.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_6_visit_g_101836'] = np.argmax(y_sigmoid_6_visit_g_101836, axis=-1) #Permet de recuperer la classe d'apartenance

## Band R

In [ ]:
nbre_6_visit_r=[96898, 103166, 105108, 106532, 108692, 111352]

In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_6_visit_r, nbre_data = nbre_data)
df_train_6_visit_r, df_test_6_visit_r, x_train_6_visit_r, x_test_6_visit_r, y_train_6_visit_r, y_test_6_visit_r = data.donnees(test_size=test_size)

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_93404 = retrieve_data(nbre_visit=93404, nbre_data = nbre_data)
df_train_93404, df_test_93404, x_train_93404, x_test_93404, y_train_93404, y_test_93404 = data_93404.donnees(test_size=test_size)


In [ ]:
# Create the CNNHyperModel model
model_6_visit_r = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_6_visit_r, x_test=x_test_6_visit_r, y_train=y_train_6_visit_r, y_test=y_test_6_visit_r)


In [ ]:
#Train the CNNHyperModel model
#hypermodel_6_visit_r, history_6_visit_r = model_6_visit_r.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
#hypermodel_6_visit_r.save(f'{rundir}/Compare_visit/models/model_6_visit_r.h5')

In [ ]:
hypermodel_6_visit_r = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit_r.h5')

In [ ]:
#Evaluate the model on testing set WITH SAME BAND
score_best_model_global_6_visit_r = hypermodel_6_visit_r.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_r = hypermodel_6_visit_r.predict(x_test_93404) #Table de probabilité


df_test_93404['y_pred_best_model_global_6_visit_r'] = np.argmax(y_sigmoid_6_visit_r, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_6_visit_r_90780 = hypermodel_6_visit_r.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_r_90780 = hypermodel_6_visit_r.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_r_90780'] = np.argmax(y_sigmoid_6_visit_r_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_6_visit_r_101836 = hypermodel_6_visit_r.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_r_101836 = hypermodel_6_visit_r.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_6_visit_r_101836'] = np.argmax(y_sigmoid_6_visit_r_101836, axis=-1) #Permet de recuperer la classe d'apartenance

## Band I

In [ ]:
nbre_6_visit_i=[91898, 93410, 95092, 97026, 103712]



In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_6_visit_i, nbre_data = nbre_data)
df_train_6_visit_i, df_test_6_visit_i, x_train_6_visit_i, x_test_6_visit_i, y_train_6_visit_i, y_test_6_visit_i = data.donnees(test_size=test_size)

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_90780 = retrieve_data(nbre_visit=90780)
df_train_90780, df_test_90780, x_train_90780, x_test_90780, y_train_90780, y_test_90780 = data_90780.donnees(test_size=test_size)

In [ ]:
# Create the CNNHyperModel model
model_6_visit_i = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_6_visit_i, x_test=x_test_6_visit_i, y_train=y_train_6_visit_i, y_test=y_test_6_visit_i)


In [ ]:
#Train the CNNHyperModel model
hypermodel_6_visit_i, history_6_visit_i = model_6_visit_i.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
hypermodel_6_visit_i.save(f'{rundir}/Compare_visit/models/model_6_visit_i.h5')

In [ ]:
hypermodel_6_visit_i = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit_i.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_i = hypermodel_6_visit_i.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_i = hypermodel_6_visit_i.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_i'] = np.argmax(y_sigmoid_6_visit_i, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_i = hypermodel_6_visit_i.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_i = hypermodel_6_visit_i.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_6_visit_i'] = np.argmax(y_sigmoid_6_visit_i, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_6_visit_i_90780 = hypermodel_6_visit_i.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_i_90780 = hypermodel_6_visit_i.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_i_90780'] = np.argmax(y_sigmoid_6_visit_i_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_6_visit_i_101836 = hypermodel_6_visit_i.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_i_101836 = hypermodel_6_visit_i.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_6_visit_i_101836'] = np.argmax(y_sigmoid_6_visit_i_101836, axis=-1) #Permet de recuperer la classe d'apartenance

## Band Z

In [ ]:
nbre_6_visit_z= [91922, 92502, 95084, 96222, 100824, 102426]

In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_6_visit_z, nbre_data = nbre_data)
df_train_6_visit_z, df_test_6_visit_z, x_train_6_visit_z, x_test_6_visit_z, y_train_6_visit_z, y_test_6_visit_z = data.donnees(test_size=test_size)

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_89762 = retrieve_data(nbre_visit=89762)
df_train_89762, df_test_89762, x_train_89762, x_test_89762, y_train_89762, y_test_89762 = data_89762.donnees(test_size=test_size)

In [ ]:
# Create the CNNHyperModel model
model_6_visit_z = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_6_visit_z, x_test=x_test_6_visit_z, y_train=y_train_6_visit_z, y_test=y_test_6_visit_z)


In [ ]:
#Train the CNNHyperModel model
hypermodel_6_visit_z, history_6_visit_z = model_6_visit_z.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
hypermodel_6_visit_z.save(f'{rundir}/Compare_visit/models/model_6_visit_z.h5')

In [ ]:
hypermodel_6_visit_z = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit_z.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_z = hypermodel_6_visit_z.evaluate(x_test_89762, y_test_89762, verbose=0)

y_sigmoid_6_visit_z = hypermodel_6_visit_z.predict(x_test_89762) #Table de probabilité


df_test_89762['y_pred_best_model_global_6_visit_z'] = np.argmax(y_sigmoid_6_visit_z, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_z = hypermodel_6_visit_z.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_z = hypermodel_6_visit_z.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_6_visit_z'] = np.argmax(y_sigmoid_6_visit_z, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_6_visit_z_90780 = hypermodel_6_visit_z.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_z_90780 = hypermodel_6_visit_z.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_z_90780'] = np.argmax(y_sigmoid_6_visit_z_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_6_visit_z_101836 = hypermodel_6_visit_z.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_z_101836 = hypermodel_6_visit_z.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_6_visit_z_101836'] = np.argmax(y_sigmoid_6_visit_z_101836, axis=-1) #Permet de recuperer la classe d'apartenance

## Band Y

In [ ]:
nbre_6_visit_y=[97608, 102128, 104180, 105560, 105748, 108242]


In [ ]:
# Recovery of normalize data and dataframe which contains features
data = retrieve_data(nbre_visit=nbre_6_visit_y, nbre_data = nbre_data)
df_train_6_visit_y, df_test_6_visit_y, x_train_6_visit_y, x_test_6_visit_y, y_train_6_visit_y, y_test_6_visit_y = data.donnees(test_size=test_size)

In [ ]:
# Recovery of normalize data and dataframe which contains features
data_110398 = retrieve_data(nbre_visit=110398)
df_train_110398, df_test_110398, x_train_110398, x_test_110398, y_train_110398, y_test_110398 = data_110398.donnees(test_size=test_size)

In [ ]:
# Create the CNNHyperModel model
model_6_visit_y = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, hyperband_max_epochs=HYPERBAND_MAX_EPOCHS,
                      max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL,
                        x_train = x_train_6_visit_y, x_test=x_test_6_visit_y, y_train=y_train_6_visit_y, y_test=y_test_6_visit_y)


In [ ]:
#Train the CNNHyperModel model
hypermodel_6_visit_y, history_6_visit_y = model_6_visit_y.Entrainement_bayes()

In [ ]:
rundir = '/sps/lsst/users/rbonnetguerrini/bogus_detection_v10/'


In [ ]:
hypermodel_6_visit_y.save(f'{rundir}/Compare_visit/models/model_6_visit_y.h5')

In [ ]:
hypermodel_6_visit_y = tf.keras.models.load_model(f'{rundir}Compare_visit/models/model_6_visit_y.h5')

In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_y = hypermodel_6_visit_y.evaluate(x_test_110398, y_test_110398, verbose=0)

y_sigmoid_6_visit_y = hypermodel_6_visit_y.predict(x_test_110398) #Table de probabilité


df_test_110398['y_pred_best_model_global_6_visit_y'] = np.argmax(y_sigmoid_6_visit_y, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on testing set
score_best_model_global_6_visit_y = hypermodel_6_visit_y.evaluate(x_test_93404, y_test_93404, verbose=0)

y_sigmoid_6_visit_y = hypermodel_6_visit_y.predict(x_test_93404) #Table de probabilité
df_test_93404['y_pred_best_model_global_6_visit_y'] = np.argmax(y_sigmoid_6_visit_y, axis=-1) #Permet de recuperer la classe d'apartenance


In [ ]:
#Evaluate the model on the visit 90780 in order to compare with train on 4 visit
score_best_model_global_6_visit_y_90780 = hypermodel_6_visit_y.evaluate(x_test_90780, y_test_90780, verbose=0)

y_sigmoid_6_visit_y_90780 = hypermodel_6_visit_y.predict(x_test_90780) #Table de probabilité
df_test_90780['y_pred_best_model_global_6_visit_y_90780'] = np.argmax(y_sigmoid_6_visit_y_90780, axis=-1) #Permet de recuperer la classe d'apartenance

In [ ]:
#Evaluate the model on the visit 101836 in order to compare with train on 4 visit
score_best_model_global_6_visit_y_101836 = hypermodel_6_visit_y.evaluate(x_test_101836, y_test_101836, verbose=0)

y_sigmoid_6_visit_y_101836 = hypermodel_6_visit_y.predict(x_test_101836) #Table de probabilité
df_test_101836['y_pred_best_model_global_6_visit_y_101836'] = np.argmax(y_sigmoid_6_visit_y_101836, axis=-1) #Permet de recuperer la classe d'apartenance

## Comparison all filter

In [ ]:
#test on their own band
df_test_90752['y_pred_best_model_global_6_visit_g']
df_test_93404['y_pred_best_model_global_6_visit_r']
df_test_90780['y_pred_best_model_global_6_visit_i']
df_test_89762['y_pred_best_model_global_6_visit_z'] 
df_test_110398['y_pred_best_model_global_6_visit_y']

# Evaluation 

In [ ]:
multiple_band_visits = [95092, 91556, 101500, 104516]


In [ ]:
model_ = 'model_4_visit'
evaluation_4_visit = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_4_visit.full_evaluation_plot()

In [ ]:
model_ = 'model_6_visit'
evaluation_6_visit = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_6_visit.full_evaluation_plot()

In [ ]:
model_ = 'model_12_visit'
evaluation_12_visit = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_12_visit.full_evaluation_plot()

In [ ]:
model_ = 'model_42_visit'
evaluation_42_visit = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_42_visit.full_evaluation_plot()

In [ ]:
model_ = 'model_all_gen3'
evaluation_gen3_visit = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_gen3_visit.full_evaluation_plot()

In [ ]:
model_ = 'model_100764'
evaluation_100764 = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_100764.full_evaluation_plot()

In [ ]:
model_ = 'model_101836'
evaluation_101836 = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_101836.full_evaluation_plot()

In [ ]:
model_ = 'model_89762'
evaluation_89762 = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_89762.full_evaluation_plot()

In [ ]:
model_ = 'model_90780'
evaluation_90780 = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_90780.full_evaluation_plot()

In [ ]:
model_ = 'model_90780_removed'
evaluation_90780_removed = evaluating_model(model=model_, visit=multiple_band_visits)
evaluation_90780_removed.full_evaluation_plot()